<a href="https://colab.research.google.com/github/jwang44/Try-colabing-in-colab/blob/main/credit_card_new_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ECSE 551 Mini-project 1
*Group 10: Junhao Wang, Yinan Zhou, and Ruilin Ji*

This notebook is dedicated for the credit card dataset, including the model, cross validation, and various experiment. 

## Start here

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd "/content/drive/My Drive/"

/content/drive/My Drive


In [3]:
import numpy as np
import pandas as pd
import scipy.stats
import statistics

## Credit Card Dataset

In [4]:
# generate new feature by multiplication and normalize
def newfeature(x,y):
  z=x*y
  norz=scipy.stats.zscore(z, axis=0, ddof=0, nan_policy='propagate')
  return norz

In [5]:
# convert csv to dataframe
df = pd.read_csv('creditcard.csv')
original_data = df.to_numpy()

In [6]:
# normalize feature
NorData = scipy.stats.zscore(df.iloc[:,:-1], axis=0, ddof=0, nan_policy='propagate') # no class column
# normalized dataset
NorDataset = np.column_stack((NorData,df.iloc[:,-1]))

New feature

In [7]:
NewF1 = newfeature(df.V3, df.V7)
NewF2 = newfeature(df.V11, df.V12)
NewF3 = newfeature(df.V12, df.V16)
NewF4 = newfeature(df.V16, df.V17)
NewF5 = newfeature(df.V16, df.V18)
NewF6 = newfeature(df.V17, df.V18)
NewF7 = df.iloc[:,0]              # initialize new feature 7 using 1st feature
n_row,n_col = np.shape(NorData)
NewFSq = np.zeros(n_row)            # initialize new features using 0s
for col in range(n_col):
  # new feature 7: multiplying all features
  if col>0:
    NewF7 = newfeature(NewF7,df.iloc[:,col])
  # new feature 8-: square all columns
  new = newfeature(df.iloc[:,col],df.iloc[:,col]) # square feature
  NewFSq = np.column_stack((NewFSq,new))

NewFSq = np.delete(NewFSq,0,1)
# new feature
NewF = np.column_stack((NewF1,NewF2,NewF3,NewF4,NewF5,NewF6,NewF7))
# NewF = np.column_stack((NewF1,NewF2,NewF3,NewF4,NewF5,NewF6))

In [8]:
# normalized dataset with new feature
NorDatasetNew = np.column_stack((NorData,NewF,df.iloc[:,-1]))

## The model

In [9]:
'''# sigmoid function
def sigmoid(a):
  # to avoid overflow in exp
  if -a > np.log(np.finfo(type(a)).max):
    return 0.0  
  return 1/(1+np.exp(-a))'''
  
# sigmoid function
def sigmoid(a):
  return 1/(1+np.exp(-a))


class Logistic_regression():
  def __init__(self):#,X_train,y_train,learning_rate,X_test,y_test):
    pass
    
  # training
  def fit(self,X_train,y_train,learning_rate):
    #n,m = np.shape(self.X_train)
    n,m = np.shape(X_train)  
    wk = np.ones([m+1,1]) # wk weights, initialized with 1
    wk1 = np.zeros([m+1,1])# wk+1 weights,initialized with 0          
    itrnum = 500       # max number of iterations 
    e = 0.001
    der = 0
    for k in range(0,itrnum):
      for i in range(0,n):
        #xi = self.X_train[i].T
        xi = X_train[i].T
        x0 = np.array([1])
        xi = np.concatenate((xi, x0),axis = 0)
        #yi = self.y_train[i]
        yi = y_train[i]
        der = der-xi*(yi-sigmoid(np.matmul(wk[:,0].T,xi))) # take derivative w.r.t w
      #wk1[:,0] = wk[:,0]-self.learning_rate*der       # update rule
      wk1[:,0] = wk[:,0]-learning_rate*der       # update rule
      if (np.linalg.norm(wk1[:,0]-wk[:,0]))**2<e:
        # print("Training stoppped early at iteration: ", k+1)      
        break 
      else:
        wk = wk1.copy()
    return wk1
  
  # validation
  def predict(self,w,X_test):
    #n,m = np.shape(self.X_test)
    n,m = np.shape(X_test)   
    y_predict = np.zeros([n,1])
    for i in range(0,n):
      #xi = self.X_test[i].T
      xi = X_test[i].T
      x0 = np.array([1])
      xi = np.concatenate((xi, x0),axis = 0)
      p1 = sigmoid(np.matmul(w.T,xi)) # calculate probabilities p(y=1|x)
      # covert probabilities to 0 or 1 by thresholding at 0.5
      if p1>=0.5:
        y_predict[i] = 1
      else:
        y_predict[i] = 0
    return y_predict

  # evaluate accuracy
  def Accu_eval(self,y_test,y_predict):
    #y_predict = self.predict(X_test)
    n,j = np.shape(y_predict)
    TP = 0;FP = 0;TN = 0;FN = 0
    # count TP,TN,FP,FN in validation set
    '''for i in range(n):
      if  self.y_test[i]==1 and y_predict[i]==1:
        TP = TP+1
      elif self.y_test[i]==0 and y_predict[i]==0:
        TN = TN+1
      elif self.y_test[i]==0 and y_predict[i]==1:
        FP = FP+1
      elif self.y_test[i]==1 and y_predict[i]==0:
        FN = FN+1'''
    for i in range(n):
      if  y_test[i]==1 and y_predict[i]==1:
        TP = TP+1
      elif y_test[i]==0 and y_predict[i]==0:
        TN = TN+1
      elif y_test[i]==0 and y_predict[i]==1:
        FP = FP+1
      elif y_test[i]==1 and y_predict[i]==0:
        FN = FN+1    
    accuracy = (TP+TN)/(TP+TN+FP+FN)
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    F = 2*precision*recall/(precision+recall)
    specificity = TN/(FP+TN)
    FPR = FP/(FP+TN)
    print("accuracy:",accuracy)
    # print("precision:",precision)
    # print("recall:",recall)
    # print("F:",F)
    # print("specificity:",specificity)
    # print("False Positive Rate:",FPR)
    # print("")
    return accuracy
    

In [10]:
# figure out which feature is of the most importance
model = Logistic_regression()
np.random.shuffle(NorDataset)
X = NorDataset[:, :-1]  # features
y = NorDataset[:, -1]   # labels
w = model.fit(X,y,learning_rate=0.001)
print(w)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in exp
  # Remove the CWD from sys.path while we load stuff.


[[ 122.47084445]
 [  94.7390446 ]
 [-405.51387301]
 [ 102.67860117]
 [  13.47884961]
 [-143.36283177]
 [ -81.09335631]
 [-226.8244388 ]
 [  16.01617032]
 [-223.52979084]
 [ -48.22147504]
 [-288.9874883 ]
 [   0.51483489]
 [ -49.17560667]
 [ -53.35039804]
 [ -58.57067771]
 [  92.42418619]
 [  12.60089495]
 [   1.10465039]
 [ -75.43775535]
 [ -57.55455013]
 [ 133.56584277]
 [  29.24635998]
 [ -49.15440343]
 [ -37.6098162 ]
 [   3.37297339]
 [ -68.78216283]
 [ -24.24531042]
 [  88.69034165]
 [ 500.17398059]]


## Cross validation

In [11]:
class Cross_validation():
  def __init__(self, k):
    # k: k-fold
    self.k = k

  def prepare_data(self, data):
    # data: np array converted from csv
    np.random.shuffle(data)
    X = data[:, :-1]  # features
    y = data[:, -1]   # labels

    # split data into k equal segments, assign them to train and test later
    Xs = np.array_split(X, self.k, axis=0)
    ys = np.array_split(y, self.k, axis=0)
    return Xs, ys

  def get_accuracy(self, Xs, ys, lr):
    accu_trains = []
    accu_tests = []
    for i in range(self.k):
      X_cv = Xs[:] # X_cross_validation
      y_cv = ys[:] # y_cross_validation

      X_test = X_cv.pop(i)
      y_test = y_cv.pop(i)

      X_train = np.concatenate(X_cv)
      y_train = np.concatenate(y_cv)

      model = Logistic_regression()
      w = model.fit(X_train, y_train, lr)

      print("----------FOLD ", i+1, "----------")

      print("----Train----")
      y_predict_train = model.predict(w, X_train)
      accu_train = model.Accu_eval(y_train, y_predict_train)
      accu_trains.append(accu_train)

      print("----Validation----")
      y_predict_test = model.predict(w, X_test)
      accu_test = model.Accu_eval(y_test, y_predict_test)
      accu_tests.append(accu_test)

    return np.mean(accu_trains), np.mean(accu_tests)


## Experiment with different learning rates

In [12]:
lrs = np.logspace(-5, -1, 10) # different learning rates to try
# or we can also try other hyperparameters here
cv = Cross_validation(10) # 10-fold cross-validation
Xs, ys = cv.prepare_data(NorDataset)
for lr in lrs:
  print("---------------LEARNING RATE: ", lr, "---------------")
  accu_train_avg, accu_val_avg = cv.get_accuracy(Xs, ys, lr)
  print("---------------TRAIN AVERAGE ACCURACY", accu_train_avg, "---------------")
  print("---------------VALIDATION AVERAGE ACCURACY", accu_val_avg, "---------------")
  print("\n-------------------------------------------------------------------------------\n")


---------------LEARNING RATE:  1e-05 ---------------
----------FOLD  1 ----------
----Train----
accuracy: 0.24803591470258138
----Validation----
accuracy: 0.23
----------FOLD  2 ----------
----Train----
accuracy: 0.24551569506726456
----Validation----
accuracy: 0.25252525252525254
----------FOLD  3 ----------
----Train----
accuracy: 0.24551569506726456
----Validation----
accuracy: 0.25252525252525254
----------FOLD  4 ----------
----Train----
accuracy: 0.25
----Validation----
accuracy: 0.21212121212121213
----------FOLD  5 ----------
----Train----
accuracy: 0.24103139013452915
----Validation----
accuracy: 0.29292929292929293
----------FOLD  6 ----------
----Train----
accuracy: 0.24439461883408073
----Validation----
accuracy: 0.26262626262626265
----------FOLD  7 ----------
----Train----
accuracy: 0.242152466367713
----Validation----
accuracy: 0.2828282828282828
----------FOLD  8 ----------
----Train----
accuracy: 0.25448430493273544
----Validation----
accuracy: 0.1717171717171717
-----

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in exp
  # Remove the CWD from sys.path while we load stuff.


----------FOLD  2 ----------
----Train----
accuracy: 0.9585201793721974
----Validation----
accuracy: 0.9595959595959596
----------FOLD  3 ----------
----Train----
accuracy: 0.9562780269058296
----Validation----
accuracy: 0.9797979797979798
----------FOLD  4 ----------
----Train----
accuracy: 0.9618834080717489
----Validation----
accuracy: 0.9292929292929293
----------FOLD  5 ----------
----Train----
accuracy: 0.9630044843049327
----Validation----
accuracy: 0.9595959595959596
----------FOLD  6 ----------
----Train----
accuracy: 0.9428251121076233
----Validation----
accuracy: 0.898989898989899
----------FOLD  7 ----------
----Train----
accuracy: 0.9596412556053812
----Validation----
accuracy: 0.9191919191919192
----------FOLD  8 ----------
----Train----
accuracy: 0.9630044843049327
----Validation----
accuracy: 0.9595959595959596
----------FOLD  9 ----------
----Train----
accuracy: 0.9585201793721974
----Validation----
accuracy: 0.9494949494949495
----------FOLD  10 ----------
----Train--

## Experiment with different features

During the experiment on different learning rates, we found that the best learning rate is **0.0002154**, so we use this learning rate for our experiment with different feature selections. 

In [ ]:
lr = 0.0002154
cv = Cross_validation(10) # 10-fold cross-validation
Xs, ys = cv.prepare_data(NorDataset)
print("----------Using normalized features, without new features----------")
accu_avg_train, accu_avg_val = cv.get_accuracy(Xs, ys, lr)
print("----------AVERAGE ACCURACY: train-",accu_avg_train," vs. validation-",accu_avg_val, "----------")
print("\n---------------------------------------------------------------------")


----------Using normalized features, without new features----------


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in exp
  # Remove the CWD from sys.path while we load stuff.


----------FOLD  1 ----------
----Train----
accuracy: 0.9528619528619529
----Validation----
accuracy: 0.93
----------FOLD  2 ----------
----Train----
accuracy: 0.9641255605381166
----Validation----
accuracy: 0.898989898989899
----------FOLD  3 ----------
----Train----
accuracy: 0.9573991031390134
----Validation----
accuracy: 0.9595959595959596
----------FOLD  4 ----------
----Train----
accuracy: 0.9562780269058296
----Validation----
accuracy: 0.9090909090909091
----------FOLD  5 ----------
----Train----
accuracy: 0.9596412556053812
----Validation----
accuracy: 0.9797979797979798
----------FOLD  6 ----------
----Train----
accuracy: 0.9495515695067265
----Validation----
accuracy: 0.9696969696969697
----------FOLD  7 ----------
----Train----
accuracy: 0.9473094170403588
----Validation----
accuracy: 0.9292929292929293
----------FOLD  8 ----------
----Train----
accuracy: 0.9562780269058296
----Validation----
accuracy: 0.9191919191919192
----------FOLD  9 ----------
----Train----
accuracy: 0.

In [ ]:
lr = 0.0002154
cv = Cross_validation(10) # 5-fold cross-validation
Xs, ys = cv.prepare_data(NorDatasetNew)
print("----------Using normalized features, with new features----------")
accu_avg_train, accu_avg_val = cv.get_accuracy(Xs, ys, lr)
print("----------AVERAGE ACCURACY: train-",accu_avg_train," vs. validation-",accu_avg_val, "----------")
print("\n---------------------------------------------------------------------")


----------Using normalized features, with new features----------
----------FOLD  1 ----------
----Train----
accuracy: 0.9562289562289562
----Validation----
accuracy: 0.97
----------FOLD  2 ----------
----Train----
accuracy: 0.9596412556053812
----Validation----
accuracy: 0.9797979797979798
----------FOLD  3 ----------
----Train----
accuracy: 0.9596412556053812
----Validation----
accuracy: 0.898989898989899
----------FOLD  4 ----------
----Train----
accuracy: 0.9618834080717489
----Validation----
accuracy: 0.9292929292929293
----------FOLD  5 ----------
----Train----
accuracy: 0.9630044843049327
----Validation----
accuracy: 0.9393939393939394
----------FOLD  6 ----------
----Train----
accuracy: 0.9618834080717489
----Validation----
accuracy: 0.9696969696969697
----------FOLD  7 ----------
----Train----
accuracy: 0.9562780269058296
----Validation----
accuracy: 0.9494949494949495
----------FOLD  8 ----------
----Train----
accuracy: 0.9607623318385651
----Validation----
accuracy: 0.9595959

## Measure the run time
See whether the model converges faster on normalized data than on original data. This is measured by training the model and time it. This part stands on its own, and is not related to any of the above process

This model class is almost the same as the one before. The only difference is that this model **does not have early stopping**, which can interfere with our timing. Also, this model is only for timing the training process, so it only has a 'fit' method

**Should we run this? Should we have early stopping or not?**

###Comparison on Original dataset and normalized dataset

In [ ]:
model = Logistic_regression()
np.random.shuffle(original_data)
X = original_data[:, :-1]  # features
y = original_data[:, -1]   # labels

In [ ]:
%%timeit
w = model.fit(X, y, learning_rate=0.001)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in exp
  # Remove the CWD from sys.path while we load stuff.


1 loop, best of 3: 6.79 s per loop


In [ ]:
model = Logistic_regression()
np.random.shuffle(NorDataset)
X = NorDataset[:, :-1]  # features
y = NorDataset[:, -1]   # labels

In [ ]:
%%timeit
w = model.fit(X, y, learning_rate=0.001)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in exp
  # Remove the CWD from sys.path while we load stuff.


1 loop, best of 3: 6.11 s per loop


###Using normalized features

In [ ]:
model = Logistic_regression()
np.random.shuffle(NorDataset)
X = NorDataset[:, :-1]  # features
y = NorDataset[:, -1]   # labels

In [ ]:
import time
import matplotlib.pyplot as plt
lr = [1]
t = []
for index in range(10):
  lr.append(lr[-1]*0.1)
  t1 = time.time()
  model.fit(X, y, learning_rate=lr[-1])
  t2 = time.time()
  print("lr: ",lr[-1],"time: ", t2-t1)
  t.append(t2-t1)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in exp
  # Remove the CWD from sys.path while we load stuff.


lr:  0.1 time:  7.110478639602661
lr:  0.010000000000000002 time:  6.729063510894775
lr:  0.0010000000000000002 time:  6.268208026885986
lr:  0.00010000000000000003 time:  6.1151816844940186
lr:  1.0000000000000004e-05 time:  0.011788129806518555
lr:  1.0000000000000004e-06 time:  0.011670827865600586
lr:  1.0000000000000005e-07 time:  0.01789093017578125
lr:  1.0000000000000005e-08 time:  0.017074108123779297
lr:  1.0000000000000005e-09 time:  0.02828073501586914
lr:  1.0000000000000006e-10 time:  0.027080297470092773


In [ ]:
plt.plot(lr[1:],t)
plt.xscale('log')
plt.yscale('log')

## Scanning more lrs, in order to find the relation between lr and time for convergence

In [ ]:
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


# df = pd.read_csv('creditcard.csv')
# data = df.to_numpy()

# # normalize feature
# NorData = scipy.stats.zscore(df.iloc[:,:-1], axis=0, ddof=0, nan_policy='propagate') # no class column
# # normalized dataset
# NorDataset = np.column_stack((NorData,df.iloc[:,-1]))

np.random.shuffle(NorDataset)
X = NorDataset[:, :-1]  # features
y = NorDataset[:, -1]   # labels

lrs = np.logspace(-5, -1, 10) # different learning rates to try
times = []

for lr in lrs:
  model = Logistic_regression()
  print("learning rate: ", lr)
  t1 = time.time()
  w = model.fit(X, y, learning_rate=lr)
  t2 = time.time()
  print("time: ", t2-t1)
  times.append(t2-t1)

plt.plot(times)